In [ ]:
import json
import pandas as pd
import sqlalchemy 
from sqlalchemy import create_engine
from sqlalchemy.sql import text
import psycopg2 
import io
import datetime as dt
from tqdm import tqdm
import os
with open(os.getenv('USER_PATH'), 'r') as f:
    config = json.load(f)
import numpy as np

config['database'] = config['database'] 

### engines = {}
### for x in ['mkt', 'dwh_db']:
###    engines[x] = sqlalchemy.create_engine('postgresql://'+config['user']+':'+config['password']+'@'+config['host']+':'+str(config['port'])+f'/{x}')

C:\Users\PhilippPascalReicher\AppData\Local\Temp\ipykernel_4200\3248813834.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [ ]:
# Connect with Matomo API
import requests
import pandas as pd
from dotenv import load_dotenv

load_dotenv()

# Define your Matomo server and API token
matomo_server = os.getenv('MATOMO_REQUEST_URL')
api_token = os.getenv('MATOMO_API_KEY')
date = '2024-03-01,2024-03-15'

# Define the API endpoint
url = f'{matomo_server}/index.php?module=API&method=API.get&format=JSON&idSite=1&period=day&date={date}&token_auth={api_token}'

# Make the request
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()
    # Create an empty list to store reshaped data
    reshaped_data = []
    # Iterate through each date in the JSON response
    for date, values in data.items():
        # Create a new dictionary for each date
        date_data = {'date': date}
        # Add other data values as columns in the dictionary
        for key, value in values.items():
            date_data[key] = value
        # Append the dictionary to the reshaped data list
        reshaped_data.append(date_data)
    # Convert the reshaped data into a DataFrame
    df = pd.DataFrame(reshaped_data)
else:
    print("Error occurred:", response.text)

In [37]:
# Connecting with Segment API
import segment.analytics as analytics

analytics.write_key = ''

In [ ]:
# Connect to Segment 
import segment.analytics as analytics
from datetime import datetime, timedelta

client = analytics.Client(write_key=os.getenv('SEGMENT_WRITE_KEY')) # write key for Segment source: Website - JS - DEV

# Define destination in Segment 
destination = 'Indicative - Website - DEV'

# Retrieve data from Segment for the past 7 days
start_date = (datetime.now() - timedelta(days=7)).strftime('%Y-%m-%d')
end_date = datetime.now().strftime('%Y-%m-%d')

# print(dir(client)) # Find available attributes of the client object

#Get data and convert to Pandas DataFrame
data = client.export(
    destination=destination, 
    format='json',
    start=start_date,
    end=end_date
)
#df = pd.json_normalize(data)

# Save DataFrame to a CSV file
# df.to_csv('segment_data.csv', index=False)


In [ ]:
# Get data from Indicative User Segment Export API
import requests
from getpass import getpass

# Set your API key and endpoint URL
api_key = os.getenv('MPARTICLE_ANALYTICS_API_KEY')
endpoint_url = os.getenv('MPARTICLE_ANALYTICS_REQUEST_URL')

# Make a GET request to retrieve data
headers = {'Authorization': f'Bearer {api_key}'}
response = requests.get(endpoint_url, headers=headers)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    data = response.json()  # Parse the JSON response
    # Process the data as needed (e.g., save to a file, analyze, etc.)
    print("Data retrieved successfully!")
else:
    print(f"Error: {response.status_code} - {response.text}")

Error: 500 - {"message":"There was an error processing your request. It has been logged (ID 9a0476dd41a405c1)."}


In [19]:
# Connecting with Salesforce API
from simple_salesforce import Salesforce
import requests
from io import StringIO

sf = Salesforce(username='philipp.reichert@zenjob.com',password='vgr1pem!edt!wtg1JCK', security_token='6kDHQsmxM3bTjaF1gArpYHSZ')

In [ ]:
# Extracting Salesforce report data
sf_instance = os.getenv('SALESFORCE_INSTANCE_URL') # Using my.salesforce instead of lightning.force 
reportId = '00O09000009zxlFEAQ' 
export = '?isdtp=p1&export=1&enc=UTF-8&xf=csv'
sfUrl = sf_instance + reportId + export
response = requests.get(sfUrl, headers=sf.headers, cookies={'sid': sf.session_id})
download_report = response.content.decode('utf-8')
df1 = pd.read_csv(StringIO(download_report))


In [23]:
# dwh connection
dwh_engine = sqlalchemy.create_engine('postgresql://'+config['user']+':'+config['password']+'@'+config['host']+':'+str(config['port'])+'/'+config['database'])
### engines = sqlalchemy.create_engine('postgresql://'+config['user']+':'+config['password']+'@'+config['host']+':'+str(config['port'])+'/[database]')

In [15]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'C'
from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import DateRange
from google.analytics.data_v1beta.types import Dimension
from google.analytics.data_v1beta.types import Metric
from google.analytics.data_v1beta.types import FilterExpression
from google.analytics.data_v1beta.types import Filter
from google.analytics.data_v1beta.types import RunReportRequest

In [16]:
# variables
start_date = dt.date(2023,1,2) 
end_date = dt.datetime.now().date() + dt.timedelta(days=-(dt.datetime.now().weekday() + 1))
# which paid campaigns
campaign_name_strings_include = ['b2b']
campaign_name_strings_exclude = ['b2c']
# which website pages
b2b_website_pages = ['de/personal', 'en/companies', 'de/erfolgsgeschichten', 'de/ressourcen'] ### add additional b2b pages here
# how many days per pull
batch_size_days = 10
# how many lines per write
batch_size_write_lines = 1000
# target table
target_table = {
    'db':'mkt',
    'schema':'mkt',
    'table':'mkt_b2b_basis_reporting'
}

In [ ]:
# Functions to interact with GA4
def ga4_response_to_df(response):
    dim_len = len(response.dimension_headers)
    metric_len = len(response.metric_headers)
    all_data = []
    for row in response.rows:
        row_data = {}
        for i in range(0, dim_len):
            row_data.update({response.dimension_headers[i].name: row.dimension_values[i].value})
        for i in range(0, metric_len):
            row_data.update({response.metric_headers[i].name: row.metric_values[i].value})
        all_data.append(row_data)
    df = pd.DataFrame(all_data)
    return df

def run_report_df(property_id, dimensions, metrics, start_date, end_date, limit, offset):
    dimensions_ga4 = []
    for dimension in dimensions:
        dimensions_ga4.append(Dimension(name=dimension))
    metrics_ga4 = []
    for metric in metrics:
        metrics_ga4.append(Metric(name=metric))
    client = BetaAnalyticsDataClient()

    request = RunReportRequest(
        property=f"properties/{property_id}",
        dimensions=dimensions_ga4,
        metrics=metrics_ga4,
        offset=offset,
        limit=limit,
        date_ranges=[DateRange(start_date=start_date.strftime('%Y-%m-%d'),
                                end_date=end_date.strftime('%Y-%m-%d'))])

    response = client.run_report(request)
    return ga4_response_to_df(response)

def auto_paginate(property_id, dimensions, metrics, start_date, end_date):
    limit = 100000
    offset = 0
    i = 0

    # first run
    tmp = pd.DataFrame()
    tmp = run_report_df(property_id, dimensions, metrics, start_date, end_date, limit, offset)
    i = i + 1

    while len(tmp) == (i * limit):
        offset = (i * limit)
        response = run_report_df(property_id, dimensions, metrics, start_date, end_date, limit, offset)
        tmp = pd.concat([
            response,
            tmp
        ])
        i = i + 1
    # clear up this horrible date format
    try: # use try because sometimes we don't have a date. 
        tmp['date'] = pd.to_datetime(tmp['date'], format='%Y%m%d')
    except:
        pass

    # make metrics useable
    for x in metrics:
        tmp[x] = tmp[x].astype('float')
    
    return tmp


In [ ]:
# get impacted B2B campaigns
strings_include_str = ' OR '.join([f"(UPPER(campaign) LIKE '%%{y.upper()}%%')" for y in campaign_name_strings_include])
strings_exclude_str = ' OR '.join([f"(UPPER(campaign) NOT LIKE '%%{y.upper()}%%')" for y in campaign_name_strings_exclude])
q = f"SELECT DISTINCT campaign FROM cds.fact_consolidated_channel_marketing_cost WHERE ({strings_include_str}) AND ({strings_exclude_str}) AND date_dt BETWEEN '{start_date}' AND '{end_date}';"
impacted_campaigns = pd.read_sql(q, engines)
impacted_campaigns = [y for y in impacted_campaigns['campaign']]

In [ ]:
# get spend impr clicks for impacted campaigns
dates = [y.date() for y in pd.date_range(start=start_date, end=end_date)]
date_batches = []
for x in range(0,len(dates),batch_size_days):
    date_batches.append(dates[x:x+batch_size_days])
campaign_where_string = ','.join([f"'{y}'" for y in impacted_campaigns])
campaign_metrics = []
for x in tqdm(date_batches):
    q = f'''SELECT 
        *
    FROM
        cds.fact_consolidated_channel_marketing_cost
    WHERE
        date_dt BETWEEN '{min(x)}' AND '{max(x)}'
        AND campaign IN ({campaign_where_string});'''
    tmp = pd.read_sql(q, engines['dwh_db'])
    campaign_metrics.append(tmp)
campaign_metrics = pd.concat(campaign_metrics).reset_index(drop=True)
campaign_metrics = campaign_metrics[[y for y in campaign_metrics if y != 'cpc']]
campaign_metrics['date_dt'] = pd.to_datetime(campaign_metrics['date_dt']).dt.date

In [ ]:
# get b2b events
ga4_events = auto_paginate(271650907, ['date', 'eventName'], ['activeUsers', 'sessions', 'eventCount'], start_date, end_date)
ga4_events = ga4_events[ga4_events['eventName'].str.contains('b2b', case=False)].reset_index(drop=True)

In [ ]:
# get b2b pages
ga4_pages = auto_paginate(271650907, ['date', 'sessionSourceMedium', 'pagePath'], ['activeUsers', 'sessions', 'screenPageViews'], start_date, end_date)
b2b_website_pages_pagePaths = []
for x in b2b_website_pages:
    b2b_website_pages_pagePaths = b2b_website_pages_pagePaths + [y for y in ga4_pages['pagePath'] if x.upper() in y.upper()]
b2b_website_pages_pagePaths = list(set(b2b_website_pages_pagePaths))
ga4_pages = ga4_pages[ga4_pages['pagePath'].isin(b2b_website_pages_pagePaths)].reset_index(drop=True)

In [ ]:
delim = ' / '
ga4_pages.loc[ga4_pages['sessionSourceMedium'] == '(not set)', 'sessionSourceMedium'] = delim.join(['(not set)','(not set)'])
ga4_pages['sessionSource'] = [y[0] for y in ga4_pages['sessionSourceMedium'].str.split(delim)]
ga4_pages['sessionMedium'] = [y[1] for y in ga4_pages['sessionSourceMedium'].str.split(delim)]
ga4_pages.loc[ga4_pages['sessionSource'] == '(direct)', 'web_origin_type'] = 'Organic'
ga4_pages.loc[ga4_pages['sessionMedium'] == 'organic', 'web_origin_type'] = 'Organic'
ga4_pages.loc[ga4_pages['sessionSourceMedium'] == '(not set)', 'web_origin_type'] = 'Organic'
ga4_pages.loc[ga4_pages['web_origin_type'].isna(), 'web_origin_type'] = 'Paid'

In [ ]:
# get organic metrics
organic_where_string = ' OR '.join([f"page LIKE '%%{y}%%'" for y in b2b_website_pages])
organic_metrics = []
for x in tqdm(date_batches):
    q = f'''SELECT 
        *
    FROM
        cds.fact_organic_search
    WHERE
        "date" BETWEEN '{min(x)}' AND '{max(x)}'
        AND ({organic_where_string});'''
    tmp = pd.read_sql(q, engines['dwh_db'])
    organic_metrics.append(tmp)
organic_metrics = pd.concat(organic_metrics).reset_index(drop=True)

In [ ]:
ga4_pages.head(5)

In [ ]:
df = pd.concat(
    [
        ga4_events.groupby(['date','eventName'], as_index=False).agg({'eventCount':'sum'}).rename(columns={'date':'date_dt'}),
        organic_metrics.groupby(['date'], as_index=False).agg({y:'sum' for y in ['impressions', 'clicks']}).rename(columns={y:'b2b_organic_'+y for y in ['impressions', 'clicks']}).rename(columns={'date':'date_dt'}),
        campaign_metrics.groupby(['date_dt'], as_index=False).agg({y:'sum' for y in ['ad_cost', 'impressions', 'clicks']}).rename(columns={
        'ad_cost':'b2b_spend',
        'impressions':'b2b_paid_impr',
        'clicks':'b2b_paid_clicks'
        }),
        ga4_pages.groupby(['date', 'web_origin_type'], as_index=False).agg({'screenPageViews':'sum'}).rename(columns={'date':'date_dt', 'screenPageViews':'ga4_pageviews'})
    ]
)
df['date_dt'] = pd.to_datetime(df['date_dt']).dt.date
# dims should go at the front so we don't look like lunatics
dims = ['date_dt', 'eventName', 'web_origin_type']
metrics = [y for y in df.columns if (y not in dims)]
df = df[dims + metrics]

In [ ]:
blocks_to_write = []
for x in range(0,len([y for y in df.index]), batch_size_write_lines):
    blocks_to_write.append(df[x:x+batch_size_write_lines])

In [ ]:
# create table 
df.head(0).to_sql(target_table['table'], engines[target_table['db']], schema=target_table['schema'], if_exists='replace',index=False, method='multi',chunksize=10000)
for x in tqdm(blocks_to_write):
    x.to_sql(target_table['table'], engines[target_table['db']], schema=target_table['schema'], if_exists='append',index=False, method='multi',chunksize=10000)

In [ ]:
df['b2b_spend'].sum()